# K- Nearest Neighbors (KNN)

In [ ]:
import logging

import pandas as pd
import sklearn
import sklearn.impute
from IPython.display import display

import vaep
import vaep.model
import vaep.models as models
import vaep.nb
from vaep import sampling
from vaep.io import datasplits
from vaep.models import ae

logger = vaep.logging.setup_logger(logging.getLogger('vaep'))
logger.info("Experiment 03 - Analysis of latent spaces and performance comparisions")

figures = {}  # collection of ax or figures

In [ ]:
# catch passed parameters
args = None
args = dict(globals()).keys()

Papermill script parameters:

In [ ]:
# files and folders
folder_experiment: str = 'runs/example'  # Datasplit folder with data for experiment
folder_data: str = ''  # specify data directory if needed
file_format: str = 'csv'  # file format of create splits, default pickle (pkl)
# Machine parsed metadata from rawfile workflow
fn_rawfile_metadata: str = 'data/dev_datasets/HeLa_6070/files_selected_metadata_N50.csv'
# training
epochs_max: int = 50  # Maximum number of epochs
# early_stopping:bool = True # Wheather to use early stopping or not
batch_size: int = 64  # Batch size for training (and evaluation)
cuda: bool = True  # Whether to use a GPU for training
# model
neighbors: int = 3  # number of neigherst neighbors to use
force_train: bool = True  # Force training when saved model could be used. Per default re-train model
sample_idx_position: int = 0  # position of index which is sample ID
model: str = 'KNN'  # model name
model_key: str = 'KNN'  # potentially alternative key for model (grid search)
save_pred_real_na: bool = True  # Save all predictions for missing values
# metadata -> defaults for metadata extracted from machine data
meta_date_col: str = None  # date column in meta data
meta_cat_col: str = None  # category column in meta data

Some argument transformations

In [ ]:
args = vaep.nb.get_params(args, globals=globals())
args = vaep.nb.args_from_dict(args)
args

Some naming conventions

In [ ]:
TEMPLATE_MODEL_PARAMS = 'model_params_{}.json'

## Load data in long format

In [ ]:
data = datasplits.DataSplits.from_folder(args.data, file_format=args.file_format)

data is loaded in long format

In [ ]:
data.train_X.sample(5)

load meta data for splits

In [ ]:
if args.fn_rawfile_metadata:
    df_meta = pd.read_csv(args.fn_rawfile_metadata, index_col=0)
    display(df_meta.loc[data.train_X.index.levels[0]])
else:
    df_meta = None

## Initialize Comparison

In [ ]:
freq_feat = sampling.frequency_by_index(data.train_X, 0)
freq_feat.head()  # training data

### Simulated missing values

The validation fake NA is used to by all models to evaluate training performance.

In [ ]:
val_pred_fake_na = data.val_y.to_frame(name='observed')
val_pred_fake_na

In [ ]:
test_pred_fake_na = data.test_y.to_frame(name='observed')
test_pred_fake_na.describe()

## Data in wide format

In [ ]:
data.to_wide_format()
args.M = data.train_X.shape[-1]
data.train_X.head()

## Train
model = 'sklearn_knn'

In [ ]:
knn_imputer = sklearn.impute.KNNImputer(n_neighbors=args.neighbors).fit(data.train_X)

### Predictions

- data of training data set and validation dataset to create predictions is the same as training data.
- predictions include missing values (which are not further compared)

create predictions and select for split entries

In [ ]:
pred = knn_imputer.transform(data.train_X)
pred = pd.DataFrame(pred, index=data.train_X.index, columns=data.train_X.columns).stack()
pred

In [ ]:
val_pred_fake_na[args.model_key] = pred
val_pred_fake_na

In [ ]:
test_pred_fake_na[args.model_key] = pred
test_pred_fake_na

save missing values predictions

In [ ]:
if args.save_pred_real_na:
    pred_real_na = ae.get_missing_values(df_train_wide=data.train_X,
                                         val_idx=val_pred_fake_na.index,
                                         test_idx=test_pred_fake_na.index,
                                         pred=pred)
    display(pred_real_na)
    pred_real_na.to_csv(args.out_preds / f"pred_real_na_{args.model_key}.csv")

### Plots

- validation data

## Comparisons


### Validation data

- all measured (identified, observed) peptides in validation data

> Does not make to much sense to compare collab and AEs,
> as the setup differs of training and validation data differs

In [ ]:
# papermill_description=metrics
d_metrics = models.Metrics()

The fake NA for the validation step are real test data (not used for training nor early stopping)

In [ ]:
added_metrics = d_metrics.add_metrics(val_pred_fake_na, 'valid_fake_na')
added_metrics

### Test Datasplit

Fake NAs : Artificially created NAs. Some data was sampled and set
explicitly to misssing before it was fed to the model for
reconstruction.

In [ ]:
added_metrics = d_metrics.add_metrics(test_pred_fake_na, 'test_fake_na')
added_metrics

Save all metrics as json

In [ ]:
vaep.io.dump_json(d_metrics.metrics, args.out_metrics / f'metrics_{args.model_key}.json')
d_metrics

In [ ]:
metrics_df = models.get_df_from_nested_dict(d_metrics.metrics,
                                            column_levels=['model', 'metric_name']).T
metrics_df

## Save predictions

In [ ]:
# save simulated missing values for both splits
val_pred_fake_na.to_csv(args.out_preds / f"pred_val_{args.model_key}.csv")
test_pred_fake_na.to_csv(args.out_preds / f"pred_test_{args.model_key}.csv")

## Config

In [ ]:
figures  # switch to fnames?

In [ ]:
args.n_params = 1  # the number of neighbors to consider
args.dump(fname=args.out_models / f"model_config_{args.model_key}.yaml")
args